__Задача__

- Выгрузить данные о заказах из учетной системы
- Выделить из них признаки: данные из формы-конструктора изделия, даты, информация о пользователе
- Выделить из них целевую переменную: время затраченное производство
- Очистить данные от персональной информации или коммерческой тайны
- Сохранить в файл для дальнейшего использования

__Выгрузка данных__

На предприятии задействована собственная система управления задачами. \
Подключение к ней требует потребует собственной библиотеки. \
В данном случае произведем подключение непосредственно к интерфейсу базы данных.

Затем полученные данные можно сохранить в файлы для последующего использования без подключения БД

In [1]:
import re
import os
import json
import datetime
import pandas as pd

from IPython.display import display
from IPython.core.display import HTML
from IPython.display import clear_output

from lib.awws_db_connection.fabula_database_connection import FabulaDatabaseConnection

import lib.utils as utils
import lib.gands as gands_utils

В итоговой версии работы параметры подключения доступны не будут -- воспроизвести выгрузку будет невозможно

In [2]:
exec(open("../config/encdec.py").read())

In [3]:
f = open('../config/db-config-awws.json', encoding='utf-8')

db_config = json.loads(''.join(f.readlines()))

In [4]:
db = FabulaDatabaseConnection(
    db_server_url=db_config['faburl'],
    db_name=db_config['dbname'],
    db_src=db_config['dbsrc'],
    db_login_hash=db_config['loginhash'],
    db_login=db_config['login'],
    db_login_2=db_config['login2'],
)

db_auth_token = db.login()

In [5]:
# db.debug_print_query = True

In [6]:
query = 'SELECT CURRENT_TIMESTAMP';

dbres = db.query(query)

display(dbres.get_error())
display(dbres.get_raw_result())
display(dbres.format())

None

{'tt': '[1+3ms]',
 'err': '',
 't': 0,
 'recs': 1,
 'fld': [{'Name': '', 'Size': 16, 'Type': 'D', 'fType': 135}],
 'res': [['12.08.22 3:40:01']]}

[{'': '12.08.22 3:40:01'}]

__Что происходит ниже__

- Выгрузить записи о заказе
    - стоимость заказа
    - дата поступления в производство
    - дата сдачи изделия
- Выгрузить данные о форме заказа. Ее заполняет клиент на веб-странице магазина
    - содержит поля из формы
    - информация об клиенте
    - вспомогательная информация о заказе
        - география
        - код страницы
        - и тд.

In [7]:
data = {
    'movs': {
        'id': None,
        'query': None,
        'rows': None,
        'rows_by_id': None,
    },
    'sp_props': {
        'id': None,
        'query': None,
        'rows': None,
        'rows_by_id': None,
    },
    'talk': {
        'id': None,
        'query': None,
        'rows': None,
        'rows_by_id': None,
    },
    'user_groups': {
        'id': None,
        'query': None,
        'rows': None,
        'rows_by_id': None,
    },
    'gands': {
        'id': None,
        'query': None,
        'rows': None,
        'rows_by_id': None,
    },
};

In [8]:
data['movs']['query'] = r"""
    SELECT __TOP__
        __ID__
        , mmpid
        , doc
        , Movement.sum as sum
        , Movement.sum2 as sum2
        , gs
        , gsdate
        , gsdate2
        , mmflag
        , firmcontract
        , person
    FROM Movement
    LEFT JOIN Docs ON Movement.doc1 = Docs.docid
    WHERE
        __WHERE_ID__
        AND mmflag >= '7'
        AND doc1 = doc
        AND (
               [mmid]  IN (SELECT [pid] FROM [Sp_property] WHERE [property] = 'userId')
            OR [mmpid] IN (SELECT [pid] FROM [Sp_property] WHERE [property] = 'userId')
        )
        AND CAST('2022-01-01T00:00:00' AS datetime) < [gsdate]
        AND CAST('2022-08-01T00:00:00' AS datetime) > [gsdate]
    __ORDER_BY__
"""

data['movs']['id'] = 'mmid'

In [9]:
data['sp_props']['query'] = r"""
    SELECT __TOP__
        __ID__, pid, property, [value]
    FROM Sp_property
    WHERE
        __WHERE_ID__
        AND pid IN (
            SELECT mmid
            FROM Movement
            WHERE
                    CAST('2022-01-01T00:00:00' AS datetime) < [gsdate]
                AND CAST('2022-08-01T00:00:00' AS datetime) > [gsdate]
                AND mmflag >= '7'
                AND doc1 = doc
        )
        AND pid IN (SELECT [pid] FROM [Sp_property] WHERE [property] = 'userId')
    __ORDER_BY__
"""

data['sp_props']['id'] = 'uid'

In [10]:
data['talk']['query'] = r"""
    SELECT __TOP__
        __ID__, dt, txt, mm, part
    FROM talk
    WHERE
        __WHERE_ID__
        AND mm IN (SELECT [pid] FROM [Sp_property] WHERE [property] = 'userId')
        AND CAST('2022-01-01T00:00:00' AS datetime) < [dt]
        AND CAST('2022-08-01T00:00:00' AS datetime) > [dt]
        AND txt LIKE '%фаза%'
    __ORDER_BY__
"""

data['talk']['id'] = 'talkid'

In [11]:
data['user_groups']['query'] = r"""
    SELECT __TOP__
        __ID__, pid, value
    FROM Property
    WHERE
        __WHERE_ID__
        AND property = 'sp Группа'
        AND pid  IN (
            SELECT [value]
            FROM Sp_property
            WHERE
                property = 'userId'
        )
    __ORDER_BY__
"""

data['user_groups']['id'] = 'uid'

In [12]:
data['gands']['query'] = r"""
    SELECT __TOP__ __ID__, gsid FROM [Gands] WHERE __WHERE_ID__ __ORDER_BY__
"""

data['gands']['id'] = 'id'

In [13]:
for k in data:
    query = data[k]['query']
    idf = data[k]['id']

    dbres = db.query(query, chunked=True, identity_field=idf)
    
    err = dbres.get_error()

    print(k + ': ' + str(err or 'ok'))

    rows = dbres.format()
    rows_by_id = dict()
    
    data[k]['rows_by_id'] = utils.keyby(rows, idf)
    data[k]['rows'] = rows
    data[k]['err'] = err

movs: ok
sp_props: ok
talk: ok
user_groups: ok
gands: ok


In [14]:
data['user_groups']['rows_by_user_id'] = utils.keyby(data['user_groups']['rows'], 'pid')

In [15]:
# печать отключена: может содержать персональные данные

# for k in data:
#     df = pd.DataFrame(data[k]['rows'])
#     display(k)
#     display(df)

Описание таблиц. \
Схема похожа на EAV.

__movement__ -- движения/задачи
- mmid -- уникальный идентификатор задачи (движения)
- mmpid -- ссылка на родительскую задачу / признак подчиненной задачи
    - movement.mmpid --> movement.mmid
- sum -- сумма
- sum2 -- общая сумма, включая подчиненные задачи
- gs -- код номенклатуры
- gsdate -- дата-время создания задачи
- gsdate2 -- дата-время готовности задачи
- mmflag -- статус (фаза)

__sp_property__, __property__ -- мета-записи
- uid -- уникальный идентификатор мета-данных
- pid -- идентификатор задачи
    - внешний ключ movement.mmid --> sp_property.pid
- property -- ключ мета записи
- value -- значение мета записи

__talk__ -- обсуждения к задачам
- talkid -- уникальный идентификатор
- dt -- датавремя создания записи
- txt -- текст сообщения
- mm -- идентификатор задачи
    - внешний ключ movement.mmid --> talk.mm
- part -- номер части (если сообщение разбито на части)

__gands__ -- номенклатура
- id -- числовой идентификатор
- gsid -- строчный идентификатор

К сожалению, из-за особенностей бизнес-процессов даты в учетной системе могут быть записаны без указания минут. \
Для того чтобы получить время поступления заказа и время его сдачи клиенту, обратимся в таблицу обсуждения задач (talks). \
В обсуждении хранится переписка и уведомления системы об переключении статусов задач. \
Переключение статусов содержит время с точностью до секунды. \
Полученную информацию можно распарсить и включить в прогноз.

- Прочитать обсуждение
- Распарсить фазы задач из обсуждения
- Связать их с задачами

In [16]:
for talk_row in data['talk']['rows']:
    mmid = talk_row['mm']
    mov_row = data['movs']['rows_by_id'].get(mmid, None)

    if (not mov_row):
        continue
        
    _mmflag_history = mov_row.get('_mmflag_history', list())
    mov_row['_mmflag_history'] = _mmflag_history
    
    # talk_row['txt'] = 'Фаза: 5 &rArr; 7п + прочий текст'
    try:
        mmflag_change_txt = re.findall(r'Фаза: \d\w? &rArr; \d\w?', talk_row['txt'])
        mmflag_change_txt = ''.join(mmflag_change_txt)
        
        if mmflag_change_txt:
            _from, _to = re.findall(r'\d\w?', mmflag_change_txt)
            _mmflag_history.append({
                'dt': talk_row['dt'],
                'from': _from,
                'to': _to,
            })

    except Exception as err:
        print(err, talk_row['txt'], re.findall(r'(\d\w?)', talk_row['txt']))

Связать мета-информацию с задачами

In [17]:
for prop_row in data['sp_props']['rows']:
    mmid = prop_row['pid']
    
    mov_row = data['movs']['rows_by_id'].get(mmid, None)

    if (not mov_row):
        continue
        
    mov_row['_sp_props'] = mov_row.get('_sp_props', list())
    
    mov_row['_sp_props'].append(prop_row)

In [18]:
for row in data['movs']['rows'][0:10]:
    print(row['mmid'])

1457017
1460692
1463859
1465559
1465560
1465568
1467195
1467196
1467211
1468875


In [19]:
# функция
# прочитать datetime (dt) из задачи
# если datetime не содержит времени (час, минуты),
# то прочитать их из текста обсуждения к задаче

def _gsdate_fix_datetime(dt, mmflags, mmflag_history):
    if dt and utils.is_contains_time(dt):
        return dt

    else:
        for mmf in mmflags:
            for row in mmflag_history:
                # статус "склад"
                if re.fullmatch(f'{mmf}', row['to']):
                    return utils.strptime(row['dt'])
                #/if
            #/for
        #/for

    return dt

Ответ из базы отсортирован по id в порядке возрастания (rows\[n-1\]\["mmid"\] < rows\[n\]\["mmid"\]) \
А значит в том же порядке, в котором они и записывались в БД \
А значит идентификаторы подчиненных задач всегда будут выше чем родительских

- Подсчитать суммы подчиненных задач
- Поправить даты
- Рассчитать время от постановки до передачи на склад (фаза 7+) (в минутах)
- Связать группу пользователей с задачами

Будем считать, что существуют такие группы: \
SP, РА, До, Ad, De

In [20]:
for mov_row in data['movs']['rows']:
    mov_row['sum2'] = mov_row['sum']

    mmid = mov_row.get('mmid', None)
    mmpid = mov_row.get('mmpid', None)

    if mmpid:
        parent_mov_row = data['movs']['rows_by_id'].get(mmpid)
        
        # Не удалось найти родительскую задачу
        if not parent_mov_row:
            print('err: not parent_mov_row:', mmid, mmpid)
            continue
            
        if parent_mov_row.get('doc') != mov_row.get('doc'):
            # print('err: pmdoc != mdoc:', mmid, mmpid)
            continue
        
        parent_mov_row['sum2'] += mov_row['sum']

    else:
        if not mov_row.get('_mmflag_history', None):
            continue

        # ------------------------
            
        # Прочитать пользователя
        userid = 0

        person = gands_utils.remove_fio_tags(mov_row.get('person', '')).strip()
        firmc = mov_row.get('firmcontract', 0)

        # Если физлицо
        if person and utils.is_numeric(person):
            userid = int(person)

        # Если контрагент
        elif firmc and utils.is_numeric(firmc) and (firmc not in [2, 3, '2', '3']):
            userid = int(firmc)

        # Если пользователь
        else:
            for prow in mov_row.get('_sp_props', []):
                if 'userId' == prow['property'] and utils.is_numeric(prow['value']):
                    userid = int(prow['value'])
                    break

        # Не удалось установить пользователя
        if not userid:
            print('err: not userid')
            continue

        mov_row['_userid'] = userid 
            
        # ------------------------
            
        # Связать пользователя с группой
        user_groups = data['user_groups']['rows_by_user_id'].get(userid, None)

        if user_groups:
            mov_row['_user_groups'] = list(map(lambda s: s.strip(), user_groups['value'].split(';')))
            
        # ------------------------

        # Дата поступления заказа на изделие
        gsdate_dt = utils.strptime(mov_row['gsdate'])

        # Дата передачи готового изделия на склад
        gsdate2_dt = utils.strptime(mov_row['gsdate2'])

        gsdate_dt = _gsdate_fix_datetime(gsdate_dt, ['1', '2', '3', '4'], mov_row['_mmflag_history'])
        gsdate2_dt = _gsdate_fix_datetime(gsdate2_dt, ['7к', '7п', '7с', '7'], mov_row['_mmflag_history'])

        mov_row['_gsdate_dt'] = gsdate_dt
        mov_row['_gsdate2_dt'] = gsdate2_dt

        # Не удалось установить дату -- пропустить
        if (not gsdate_dt) or (not gsdate2_dt):
            print('err: (not gsdate_dt) or (not gsdate2_dt):', mmid, gsdate_dt, gsdate2_dt)
            continue;

        # Длительность производства в минутах
        prod_t = abs((gsdate2_dt - gsdate_dt).total_seconds() // 60)

        mov_row['_prod_t'] = prod_t    

err: not parent_mov_row: 1470251 1470246
err: not parent_mov_row: 1471252 1468272
err: (not gsdate_dt) or (not gsdate2_dt): 1489616 2022-01-21 10:02:58 None
err: (not gsdate_dt) or (not gsdate2_dt): 1520975 2022-03-04 00:00:00 None
err: not parent_mov_row: 1527954 1527953
err: not parent_mov_row: 1536159 1536157
err: not parent_mov_row: 1559479 1559701
err: not parent_mov_row: 1562461 1564433
err: (not gsdate_dt) or (not gsdate2_dt): 1562974 2022-06-21 17:01:00 None
err: not parent_mov_row: 1570006 1571945
err: not parent_mov_row: 1573793 1580690
err: not parent_mov_row: 1573808 1580690
err: not parent_mov_row: 1585001 1585000
err: not parent_mov_row: 1587398 1591848
err: not parent_mov_row: 1588339 1599427
err: (not gsdate_dt) or (not gsdate2_dt): 1608417 2022-06-27 10:32:00 None
err: not parent_mov_row: 1615796 1256537
err: not parent_mov_row: 1616467 1616522
err: not parent_mov_row: 1616475 1616522
err: not parent_mov_row: 1620624 1620623
err: not parent_mov_row: 1620635 1620623
err

Сформировать окончательный вариант учебных данных
- Исключить подчиненные задачи
- Исключить задачи без времени производства
- Исключить задачи без мета-данных из формы
- Исключить задачи без группы пользователей
- Исключить персональные данные и "мусор"

Признаки
- мета-данные
- код номенклатуры
- группы в которых состоит пользователь

Целевые переменные
- prod_t -- время производства
- sum -- цена

Результат -- массив объектов с неопределенным составом полей
То есть строки могут содержать различные поля

Итоговые данные должны быть обезличены

In [21]:
_props_to_drop = {
    '0': 1,
    '__usergeo': 1,
    'загрузить макет': 1,
    'комментарий по штрих-коду': 1,
    'комментарий к заказу': 1,
    'комментарий': 1,
    '_шаблон_в_редакторе': 1,
    '_категория_в_редакторе': 1,
    'лицевая сторона': 1,
    'оборотная сторона': 1,
    'файлы': 1,
    'пункт выдачи': 1,
    'пожелания в дизайне': 1,
    'рабочее название': 1,
    'текст для бирки': 1,
    'файл для записи на магнитную ленту': 1,
    'tel': 1,
    'email': 1,
    'userId': 1,
    'template_id': 1,
}

_props_to_enc = {
    # ...
}

# SP, РА, До, Ad, De
_user_groups_mask = {
    'SP': 0b_0000_0001,
    'РА': 0b_0000_0010,
    'До': 0b_0000_0100,
    'Ad': 0b_0000_1000,
    'De': 0b_0001_0000,
}

def _is_valid_prop_row(row):
    t = type(row['value'])
    v = row['value']
    k = row['property']
    
    # если поле не строка и не число -> пропустить
    if (not (t == int or t == str)):
        return False

    # поле, вероятно, содержит файлы -> пропустить
    if '[object FileList]' == v:
        return False

    # поле, вероятно, содержит пути к файлам -> пропустить
    if t == str and os.path.isabs(v):
        return False

    # поле, вероятно, содержит пути к файлам (макропути) -> пропустить
    if utils.is_macro_path(v):
        return False
    
    # если поле содержит номера телефонов -> пропустить
    if utils.is_telephone_number(v):
        return False
    
    # если поле содержит email -> пропустить
    if utils.is_email(v):
        return False
    
    # поле, вероятно, ошибочное -> пропустить
    if len(re.findall(r'\.info', k)):
        return False
    
    if _props_to_drop.get(k):
        return False
        
    return True


final_train_data = []


for row in data['movs']['rows']:
    # пропустить подчиненные задачи
    if row.get('mmpid'):
        continue
    
    # пропустить задачи без оценки времени выполнения
    if not row.get('_prod_t'):
        continue
        
    # пропустить задачи без мета записей
    if not row.get('_sp_props'):
        continue
        
    # пропустить задачи, где не известна группа пользователя
    if not row.get('_user_groups'):
        continue
        
    _finrow = dict()
    _finrow['mmid'] = row['mmid']
    _finrow['gs'] = row['gs']
    _finrow['prod_t'] = row['_prod_t']
    _finrow['sum'] = row['sum2']
    _finrow['user_groups'] = 0
    _finrow['gsdate_ts'] = row['_gsdate_dt'].timestamp()
    _finrow['gsdate2_ts'] = row['_gsdate2_dt'].timestamp()
    _finrow['userId'] = custom_enc(int(row['_userid']))
    
    for prow in row['_sp_props']:
        if _props_to_enc.get(prow['property']):
            _finrow[prow['property']] = custom_enc(int(prow['value']))
        
        elif _is_valid_prop_row(prow):
            _finrow[prow['property']] = prow['value']
            
    # представить группы пользователей как 8-битную маску
    for user_group in row['_user_groups']:
        _finrow['user_groups'] = _finrow['user_groups'] | _user_groups_mask.get(user_group, 0)
        
    final_train_data.append(_finrow)

In [22]:
df = pd.DataFrame(final_train_data)

df

,mmid,gs,prod_t,sum,user_groups,gsdate_ts,gsdate2_ts,userId,gsid,post_id,...,цветность перекидных листов,количество листов в блоке,вид значка,тип коробки,размер коробки,цвет ленты,цвет нанесения,способ упаковки,макет основы,добавить цепочку
0,1457017,ГПШрПЛ02,8708.0,1114.0,1,1.641298e+09,1.641820e+09,12.281157,ТСИЗ3201,230,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1463859,ГППКПК,4457.0,9290.0,7,1.642162e+09,1.642429e+09,12.230668,ТСИЗ0601,221,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1465559,ГПИНПР,5231.0,3000.0,1,1.641827e+09,1.642141e+09,12.244904,ТСИЗ1601,297,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1467195,ГППКПК,3234.0,3490.0,3,1.641800e+09,1.641994e+09,12.237959,ТСИЗ0601,221,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1467211,ГППОВиФо,1998.0,3330.0,7,1.641762e+09,1.641882e+09,12.231175,ТСИЗ0201,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10223,1641007,ГПСУПОК1,391.0,448.0,1,1.659079e+09,1.659102e+09,12.287791,ГПСУПОК1,227,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10224,1641014,ГПШрПЛ02,4462.0,2544.0,7,1.659096e+09,1.659364e+09,12.223701,ГПШрПЛ,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10225,1641212,ГПШрПЛ01,4261.0,781.0,7,1.659081e+09,1.659336e+09,12.242964,ГПШрПЛ,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10226,1641446,ГПШрПЛ02,157.0,140.0,3,1.659092e+09,1.659101e+09,12.263176,ГПШрПЛ,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Состав полей разный. \
Потому, некоторые строки могут быть NaN

In [24]:
f = open('./data/crm/train.json', 'w', encoding='utf-8')
f.write(json.dumps(final_train_data, ensure_ascii=False))

4341563

Конечный вид -- массив javascript объектов

```javascript
// train.json
arr = [obj_1, obj_2, ... obj_n]
```

где
```javascript
obj = {
    mmid         : 1641007,     // идентификатор задачи/изделия/заказа
    gs           : 'ГПШрПЛ02',  // код номенклатуры-изделия
    gsid         : 'ТСИЗ3201',  // код номенклатуры-формы заказа
    gsdate_ts    : 1659964390,  // время поступления (timestamp)
    gsdate2_ts   : 1663984390,  // время сдачи (timestamp)
    prod_t       : 4462,        // время изготовления в минутах
    sum          : 2544,        // стоимость изделия
    user_groups  : 7,           // битовая-маска группы в которых состоит пользователь
    post_id      : 201,         // код страницы
    userId       : 12.012931,   // Обфусцированный идентификатор пользователя
    __источник   : 'ма',        // источник заказа (веб-магазин, внутренний заказ и проч.)
    
    // ... 
    // прочие поля
}
```

In [25]:
# каталог номенклатуры
# для подтверждения существования кодов

final_gands_data = list(map(lambda row: [row['id'], row['gsid']], data['gands']['rows']))

f = open('./data/gands/gands.json', 'w', encoding='utf-8')
f.write(json.dumps(final_gands_data, ensure_ascii=False))

196264